In [4]:
import numpy as np
import pandas as pd
import model.preprocess_games as ppg
from pandas import DataFrame

In [2]:
DATA_PATH = 'data'

SEASONS_PROCESSED_DS = f"{DATA_PATH}/seasons.processed.feather"

TEAMS_DS = f"{DATA_PATH}/teams.processed.feather"
TEAMS_PROCESSED_DS = f"{DATA_PATH}/teams.processed.feather"

GAMES_DS = f"{DATA_PATH}/games.csv"
GAMES_PROCESSED_DS = f"{DATA_PATH}/games.processed.feather"

In [7]:
games = pd.read_csv(GAMES_DS, parse_dates=["GAME_DATE_EST"]
                    , infer_datetime_format=True, index_col="GAME_ID")
games = games.sort_values(by=['GAME_DATE_EST', 'GAME_ID'])
teams = pd.read_feather(TEAMS_PROCESSED_DS)
seasons = pd.read_feather(SEASONS_PROCESSED_DS)
season_games = ppg.get_season_games(games, seasons)
games_matchup = pd.read_feather(GAMES_PROCESSED_DS)

In [12]:
games_matchup.tail(30)

,ID,DATE,HT,HT_RANK,HT_CLASS,HT_HW,HT_HL,HT_VW,HT_VL,HT_LAST10_W,HT_LAST10_L,HT_LAST10_MATCHUP_W,HT_LAST10_MATCHUP_L,HT_OVERALL_OFF_POINTS,HT_OVERALL_DEF_POINTS,HT_OVERALL_OFF_FG,HT_OVERALL_DEF_FG,HT_OVERALL_OFF_3P,HT_OVERALL_DEF_3P,HT_OVERALL_OFF_FT,HT_OVERALL_DEF_FT,HT_OVERALL_OFF_REB,HT_OVERALL_DEF_REB,HT_AWAY_POINTS,HT_AWAY_FG,HT_AWAY_3P,HT_AWAY_FT,HT_AWAY_REB,VT,VT_RANK,VT_CLASS,VT_HW,VT_HL,VT_VW,VT_VL,VT_LAST10_W,VT_LAST10_L,VT_LAST10_MATCHUP_W,VT_LAST10_MATCHUP_L,VT_OVERALL_OFF_POINTS,VT_OVERALL_DEF_POINTS,VT_OVERALL_OFF_FG,VT_OVERALL_DEF_FG,VT_OVERALL_OFF_3P,VT_OVERALL_DEF_3P,VT_OVERALL_OFF_FT,VT_OVERALL_DEF_FT,VT_OVERALL_OFF_REB,VT_OVERALL_DEF_REB,VT_AWAY_POINTS,VT_AWAY_FG,VT_AWAY_3P,VT_AWAY_FT,VT_AWAY_REB
3659,21600030,2016-10-29,1610612741,None,None,1,0,0,0,1,0,6,4,52.5,49.500,0.196,0.250,0.220,0.174,0.366,0.312,27.500,18.000,0.0,0.000,0.000,0.000,0.0,1610612754,None,None,1,0,0,1,1,1,4,6,112.0,112.0,0.442,0.435,0.409,0.339,0.708,0.786,50.5,50.5,94.0,0.378,0.292,0.650,49.0
3660,21600029,2016-10-29,1610612739,None,None,1,0,1,0,2,0,10,0,105.5,89.500,0.448,0.378,0.373,0.292,0.732,0.847,45.500,46.500,94.0,0.418,0.375,0.727,40.0,1610612753,None,None,0,1,0,1,0,2,0,10,89.0,108.0,0.365,0.495,0.320,0.309,0.726,0.729,44.5,52.5,82.0,0.347,0.308,0.667,44.0
3661,21600028,2016-10-29,1610612752,None,None,0,0,0,1,0,1,2,8,44.0,58.500,0.184,0.240,0.166,0.186,0.375,0.368,21.000,25.500,88.0,0.368,0.333,0.750,42.0,1610612763,None,None,1,0,0,0,1,0,8,2,51.0,49.0,0.225,0.210,0.229,0.166,0.352,0.345,22.5,22.5,0.0,0.000,0.000,0.000,0.0
3662,21600027,2016-10-29,1610612766,None,None,0,0,2,0,2,0,4,6,68.0,62.333,0.276,0.265,0.207,0.200,0.555,0.467,33.333,32.667,102.0,0.414,0.311,0.833,50.0,1610612738,None,None,1,0,0,1,1,1,6,4,110.5,111.0,0.520,0.417,0.346,0.390,0.729,0.766,41.5,49.5,99.0,0.500,0.348,0.625,36.0
3663,21600026,2016-10-29,1610612755,None,None,0,1,0,0,0,1,1,9,48.5,51.500,0.214,0.208,0.146,0.136,0.360,0.366,21.500,26.500,0.0,0.000,0.000,0.000,0.0,1610612737,None,None,1,0,0,0,1,0,9,1,57.0,49.5,0.250,0.213,0.231,0.150,0.389,0.361,26.0,20.0,0.0,0.000,0.000,0.000,0.0
3664,21600025,2016-10-28,1610612740,None,None,0,1,0,0,0,1,1,9,51.0,53.500,0.206,0.223,0.079,0.166,0.442,0.379,16.500,29.000,0.0,0.000,0.000,0.000,0.0,1610612744,None,None,0,1,0,0,0,1,9,1,50.0,64.5,0.236,0.240,0.106,0.250,0.361,0.442,17.5,27.5,0.0,0.000,0.000,0.000,0.0
3665,21600024,2016-10-28,1610612762,None,None,0,0,0,1,0,1,6,4,52.0,56.500,0.244,0.260,0.166,0.342,0.500,0.500,15.500,17.000,104.0,0.488,0.333,1.000,31.0,1610612747,None,None,1,0,0,0,1,0,4,6,60.0,57.0,0.253,0.256,0.172,0.121,0.409,0.352,19.5,19.5,0.0,0.000,0.000,0.000,0.0
3666,21600023,2016-10-28,1610612742,None,None,0,0,0,1,0,1,4,6,60.5,65.000,0.216,0.252,0.188,0.263,0.361,0.382,24.500,26.000,121.0,0.433,0.375,0.722,49.0,1610612745,None,None,0,0,0,1,0,1,6,4,57.0,60.0,0.256,0.253,0.121,0.172,0.352,0.409,19.5,19.5,114.0,0.512,0.241,0.704,39.0
3667,21600022,2016-10-28,1610612748,None,None,0,0,1,0,1,0,6,4,54.0,48.000,0.242,0.191,0.125,0.166,0.312,0.393,26.000,22.500,108.0,0.485,0.250,0.625,52.0,1610612766,None,None,0,0,1,0,1,0,4,6,53.5,48.0,0.224,0.212,0.152,0.094,0.416,0.375,25.5,23.0,107.0,0.449,0.304,0.833,51.0
3668,21600021,2016-10-28,1610612760,None,None,0,0,1,0,1,0,7,3,51.5,48.500,0.208,0.214,0.136,0.146,0.366,0.360,26.500,21.500,103.0,0.415,0.273,0.731,53.0,1610612756,None,None,0,1,0,0,0,1,3,7,47.0,56.5,0.234,0.256,0.125,0.143,0.334,0.366,20.5,20.5,0.0,0.000,0.000,0.000,0.0
